In [1]:
import json
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.append(str(Path.cwd().parent / 'backend'))

print("✅ Imports successful")

✅ Imports successful


## 1. Load and Validate JSON Structure

In [2]:
# Load friend item pool
pool_path = Path.cwd().parent / 'backend' / 'app' / 'data' / 'selve_friend_item_pool.json'

with open(pool_path) as f:
    friend_pool = json.load(f)

print(f"✅ Loaded friend item pool from: {pool_path}")
print(f"   Dimensions: {len(friend_pool)}")
print(f"   Total items: {sum(len(items) for items in friend_pool.values())}")

# Verify structure
expected_dimensions = ['LUMEN', 'AETHER', 'ORPHEUS', 'ORIN', 'LYRA', 'VARA', 'CHRONOS', 'KAEL']
missing = set(expected_dimensions) - set(friend_pool.keys())
if missing:
    print(f"❌ Missing dimensions: {missing}")
else:
    print(f"✅ All 8 dimensions present")

# Check item structure
required_fields = ['id', 'text', 'dimension', 'reversed', 'correlation', 'observability']
for dimension, items in friend_pool.items():
    for item in items:
        missing_fields = set(required_fields) - set(item.keys())
        if missing_fields:
            print(f"❌ {item['id']} missing fields: {missing_fields}")
            break
    else:
        continue
    break
else:
    print(f"✅ All items have required fields")

✅ Loaded friend item pool from: /home/chris/selve/backend/app/data/selve_friend_item_pool.json
   Dimensions: 8
   Total items: 27
✅ All 8 dimensions present
✅ All items have required fields


## 2. Grammar Validation

In [3]:
# Check for first-person pronouns
first_person_pronouns = [' I ', ' my ', ' myself', ' me ', 'I am', 'I have', 'I do']
second_person_pronouns = [' you ', ' your ', ' yourself', 'you are', 'you have']

grammar_issues = []

for dimension, items in friend_pool.items():
    for item in items:
        text = item['text']
        
        # Check for first-person
        for pronoun in first_person_pronouns:
            if pronoun in text:
                grammar_issues.append({
                    'id': item['id'],
                    'issue': f'First-person pronoun: "{pronoun}"',
                    'text': text
                })
                break
        
        # Check for second-person
        for pronoun in second_person_pronouns:
            if pronoun in text:
                grammar_issues.append({
                    'id': item['id'],
                    'issue': f'Second-person pronoun: "{pronoun}"',
                    'text': text
                })
                break
        
        # Check for proper {Name} usage
        if '{Name}' not in text:
            grammar_issues.append({
                'id': item['id'],
                'issue': 'Missing {Name} placeholder',
                'text': text
            })

if grammar_issues:
    print(f"❌ Found {len(grammar_issues)} grammar issues:")
    for issue in grammar_issues:
        print(f"\n{issue['id']}:")
        print(f"  Issue: {issue['issue']}")
        print(f"  Text: {issue['text'][:80]}...")
else:
    print("✅ All questions use proper third-person grammar with {Name} placeholder")

✅ All questions use proper third-person grammar with {Name} placeholder


## 3. Statistical Validation

In [4]:
import statistics

# Collect statistics
all_correlations = []
observability_counts = {'observable': 0, 'ambiguous': 0, 'internal': 0}
reversed_counts = {'reversed': 0, 'normal': 0}
items_per_dimension = {}

for dimension, items in friend_pool.items():
    items_per_dimension[dimension] = len(items)
    
    for item in items:
        all_correlations.append(item['correlation'])
        observability_counts[item['observability']] += 1
        reversed_counts['reversed' if item['reversed'] else 'normal'] += 1

# Report
print("📊 STATISTICAL SUMMARY")
print("=" * 60)

print(f"\nItems per dimension:")
for dim, count in items_per_dimension.items():
    status = "✅" if count >= 3 else "⚠️ "
    print(f"  {status} {dim}: {count} items")

print(f"\nCorrelations:")
print(f"  Mean: {statistics.mean(all_correlations):.3f}")
print(f"  Median: {statistics.median(all_correlations):.3f}")
print(f"  Min: {min(all_correlations):.3f}")
print(f"  Max: {max(all_correlations):.3f}")
print(f"  Std Dev: {statistics.stdev(all_correlations):.3f}")

total = sum(observability_counts.values())
print(f"\nObservability:")
for obs_type, count in observability_counts.items():
    pct = (count / total) * 100
    print(f"  {obs_type}: {count} ({pct:.1f}%)")

print(f"\nItem Balance:")
for rev_type, count in reversed_counts.items():
    pct = (count / total) * 100
    print(f"  {rev_type}: {count} ({pct:.1f}%)")

# Validation checks
print(f"\n🎯 VALIDATION CHECKS:")
print(f"  {'✅' if statistics.mean(all_correlations) >= 0.55 else '❌'} Average correlation >= 0.55: {statistics.mean(all_correlations):.3f}")
print(f"  {'✅' if min(items_per_dimension.values()) >= 3 else '⚠️ '} All dimensions have >= 3 items: min = {min(items_per_dimension.values())}")
print(f"  {'✅' if total >= 24 else '❌'} Total items >= 24: {total}")
print(f"  {'✅' if observability_counts['observable'] >= 8 else '⚠️ '} Observable items >= 30%: {observability_counts['observable']} ({(observability_counts['observable']/total)*100:.1f}%)")

📊 STATISTICAL SUMMARY

Items per dimension:
  ✅ LUMEN: 4 items
  ✅ AETHER: 3 items
  ✅ ORPHEUS: 3 items
  ✅ ORIN: 4 items
  ✅ LYRA: 3 items
  ✅ VARA: 3 items
  ✅ CHRONOS: 4 items
  ✅ KAEL: 3 items

Correlations:
  Mean: 0.618
  Median: 0.610
  Min: 0.515
  Max: 0.790
  Std Dev: 0.079

Observability:
  observable: 10 (37.0%)
  ambiguous: 17 (63.0%)
  internal: 0 (0.0%)

Item Balance:
  reversed: 15 (55.6%)
  normal: 12 (44.4%)

🎯 VALIDATION CHECKS:
  ✅ Average correlation >= 0.55: 0.618
  ✅ All dimensions have >= 3 items: min = 3
  ✅ Total items >= 24: 27
  ✅ Observable items >= 30%: 10 (37.0%)


## 4. Display Sample Questions

In [5]:
# Show first 2 items from each dimension
print("📋 SAMPLE QUESTIONS (First 2 per dimension)")
print("=" * 80)

for dimension, items in friend_pool.items():
    print(f"\n{dimension}:")
    for i, item in enumerate(items[:2]):
        marker = "[R]" if item['reversed'] else "[N]"
        obs = item['observability'][:3].upper()
        corr = item['correlation']
        
        # Replace {Name} with example name for readability
        example_text = item['text'].replace('{Name}', 'Sarah')
        
        print(f"\n  {marker} [{obs}] r={corr:.2f}")
        print(f"  {example_text}")

📋 SAMPLE QUESTIONS (First 2 per dimension)

LUMEN:

  [R] [OBS] r=0.79
  Sarah doesn't talk a lot. At parties, they find themselves observing conversations from the sidelines rather than jumping in.

  [R] [AMB] r=0.75
  Sarah says little.

AETHER:

  [R] [AMB] r=0.69
  Sarah gets upset easily.

  [R] [AMB] r=0.65
  Sarah gets stressed out easily. When multiple demands pile up, they feel their stress level rising faster than it seems to affect those around them.

ORPHEUS:

  [R] [AMB] r=0.61
  Sarah is not interested in other people's problems. When someone vents about their difficulties, they seem more focused on their own issues rather than showing deep concern for others.

  [N] [AMB] r=0.69
  Sarah sympathizes with others' feelings.

ORIN:

  [R] [OBS] r=0.54
  Sarah makes a mess of things. When they dive into a project, their workspace—or even their kitchen—quickly turns chaotic, with papers, tools, utensils or ingredients scattered everywhere until they’ve finished.

  [R] [AMB] 

## 5. Test Adaptive Algorithm (if available)

Test if the adaptive testing module can work with friend questions.

In [6]:
# Try to import adaptive testing
try:
    from app.adaptive_testing import AdaptiveTesting
    print("✅ Adaptive testing module imported")
    
    # Test initialization
    adaptive = AdaptiveTesting(
        item_pool=friend_pool,
        min_items_per_dimension=2,
        max_items_per_dimension=4
    )
    
    print("✅ Adaptive testing initialized with friend pool")
    print(f"   Min items per dimension: 2")
    print(f"   Max items per dimension: 4")
    print(f"   Expected total: 16-32 questions")
    
except Exception as e:
    print(f"⚠️  Could not import adaptive testing: {e}")
    print("   This is OK - we'll test it in Phase 2 integration")

⚠️  Could not import adaptive testing: cannot import name 'AdaptiveTesting' from 'app.adaptive_testing' (/home/chris/selve/backend/app/adaptive_testing.py)
   This is OK - we'll test it in Phase 2 integration


## 6. Simulate Friend Assessment Flow

In [7]:
# Simulate what a friend would see
print("🎭 SIMULATED FRIEND ASSESSMENT FLOW")
print("=" * 80)

friend_name = "John"
inviter_name = "Sarah"

print(f"\nScenario: {friend_name} is assessing {inviter_name}")
print(f"\nSample questions {friend_name} would see:\n")

# Pick 5 random questions
import random
random.seed(42)  # For reproducibility

sample_questions = []
for dimension, items in friend_pool.items():
    if items:
        sample_questions.append(random.choice(items))

sample_questions = sample_questions[:5]

for i, item in enumerate(sample_questions, 1):
    # Replace {Name} with inviter's name
    question_text = item['text'].replace('{Name}', inviter_name)
    
    print(f"Question {i}:")
    print(f"{question_text}")
    print(f"\n[ Strongly Disagree ] [ Disagree ] [ Neutral ] [ Agree ] [ Strongly Agree ] [ Not Sure ]")
    print(f"\n{'─' * 80}\n")

print("✅ Friend assessment flow simulation complete")

🎭 SIMULATED FRIEND ASSESSMENT FLOW

Scenario: John is assessing Sarah

Sample questions John would see:

Question 1:
Sarah doesn't talk a lot. At parties, they find themselves observing conversations from the sidelines rather than jumping in.

[ Strongly Disagree ] [ Disagree ] [ Neutral ] [ Agree ] [ Strongly Agree ] [ Not Sure ]

────────────────────────────────────────────────────────────────────────────────

Question 2:
Sarah gets upset easily.

[ Strongly Disagree ] [ Disagree ] [ Neutral ] [ Agree ] [ Strongly Agree ] [ Not Sure ]

────────────────────────────────────────────────────────────────────────────────

Question 3:
Sarah feels others' emotions.

[ Strongly Disagree ] [ Disagree ] [ Neutral ] [ Agree ] [ Strongly Agree ] [ Not Sure ]

────────────────────────────────────────────────────────────────────────────────

Question 4:
Sarah gets chores done right away.

[ Strongly Disagree ] [ Disagree ] [ Neutral ] [ Agree ] [ Strongly Agree ] [ Not Sure ]

─────────────────────

## 7. Completion Time Estimation

In [8]:
# Estimate completion time
total_items = sum(len(items) for items in friend_pool.values())
avg_time_per_item_seconds = 12  # Conservative estimate

min_time_minutes = (16 * avg_time_per_item_seconds) / 60  # If adaptive selects 16 items
max_time_minutes = (total_items * avg_time_per_item_seconds) / 60  # If all items shown
expected_time_minutes = (24 * avg_time_per_item_seconds) / 60  # Expected adaptive selection

print("⏱️  COMPLETION TIME ESTIMATION")
print("=" * 60)
print(f"\nTotal items available: {total_items}")
print(f"Average time per item: {avg_time_per_item_seconds} seconds")
print(f"\nExpected scenarios:")
print(f"  Minimum (adaptive, 16 items): {min_time_minutes:.1f} minutes")
print(f"  Expected (adaptive, 24 items): {expected_time_minutes:.1f} minutes")
print(f"  Maximum (all {total_items} items): {max_time_minutes:.1f} minutes")

if expected_time_minutes <= 20:
    print(f"\n✅ Expected time ({expected_time_minutes:.1f} min) is reasonable for visitors")
else:
    print(f"\n⚠️  Expected time ({expected_time_minutes:.1f} min) may be too long for visitors")

⏱️  COMPLETION TIME ESTIMATION

Total items available: 27
Average time per item: 12 seconds

Expected scenarios:
  Minimum (adaptive, 16 items): 3.2 minutes
  Expected (adaptive, 24 items): 4.8 minutes
  Maximum (all 27 items): 5.4 minutes

✅ Expected time (4.8 min) is reasonable for visitors


## 8. Final Validation Report

In [9]:
print("📋 FINAL VALIDATION REPORT")
print("=" * 80)

checks = []

# JSON validity
checks.append(("JSON structure valid", True))

# Grammar
checks.append(("All questions use third-person grammar", len(grammar_issues) == 0))

# Coverage
checks.append(("All 8 dimensions present", len(friend_pool) == 8))
checks.append(("All dimensions have >= 3 items", min(items_per_dimension.values()) >= 3))
checks.append(("Total items >= 24", total >= 24))

# Quality
checks.append(("Average correlation >= 0.55", statistics.mean(all_correlations) >= 0.55))
checks.append(("Observable items >= 30%", (observability_counts['observable']/total) >= 0.30))

# Completion time
checks.append(("Expected completion time <= 20 minutes", expected_time_minutes <= 20))

passed = sum(1 for _, result in checks if result)
total_checks = len(checks)

print(f"\nValidation Results: {passed}/{total_checks} checks passed\n")

for check_name, result in checks:
    status = "✅" if result else "❌"
    print(f"{status} {check_name}")

if passed == total_checks:
    print(f"\n🎉 ALL CHECKS PASSED! Friend item pool is ready for Phase 2.")
    print(f"\n📁 File location: {pool_path}")
    print(f"\n🚀 Next Steps:")
    print(f"   1. Proceed to Phase 2: Backend API implementation")
    print(f"   2. Create friend response submission endpoint")
    print(f"   3. Implement quality score calculation")
    print(f"   4. Build profile regeneration service")
else:
    print(f"\n⚠️  {total_checks - passed} check(s) failed. Review issues above.")

📋 FINAL VALIDATION REPORT

Validation Results: 8/8 checks passed

✅ JSON structure valid
✅ All questions use third-person grammar
✅ All 8 dimensions present
✅ All dimensions have >= 3 items
✅ Total items >= 24
✅ Average correlation >= 0.55
✅ Observable items >= 30%
✅ Expected completion time <= 20 minutes

🎉 ALL CHECKS PASSED! Friend item pool is ready for Phase 2.

📁 File location: /home/chris/selve/backend/app/data/selve_friend_item_pool.json

🚀 Next Steps:
   1. Proceed to Phase 2: Backend API implementation
   2. Create friend response submission endpoint
   3. Implement quality score calculation
   4. Build profile regeneration service
